Note: for using the widgets inside jupyterlab, you need an appropriate environment with nodejs and the widegetextensions install. 
In order to develop the notebook, the following environment was used:

- a new "empty" python 3.10 environment, for instance created with conda 
  - conda create -n secanalyzing python==3.10
  - conda activate secanalyzing
- pip install jupyterlab
- jupyter labextension install @jupyter-widgets/jupyterlab-manager
- pip install secfsdstools
- pip install ipywidgets

In [1]:
# Basic import to support interactive widgets in notebooks
import ipywidgets as widgets
from IPython.display import display, Markdown
from ipywidgets import interact, interact_manual

import pandas as pd
pd.set_option('display.max_rows', 500) # ensure that all rows are shown
pd.set_option('display.max_colwidth', None)

In [2]:
# imports from the secfsdstools package
from secfsdstools.c_index.searching import IndexSearch
from secfsdstools.c_index.companyindexreading import CompanyIndexReader
from secfsdstools.e_collector.reportcollecting import SingleReportCollector
from secfsdstools.e_filter.rawfiltering import ReportPeriodRawFilter, StmtRawFilter
from secfsdstools.e_presenter.presenting import StandardStatementPresenter
from secfsdstools.c_index.indexing import BaseReportIndexer

In [3]:
# initialize the search class
search = IndexSearch.get_index_search()

# create a list with all known forms
forms_list = sorted(search.dbaccessor.read_all_indexreports_df().form.unique().tolist())
stmt_list = ['BS', 'CF', 'CI', 'CP', 'EQ', 'IS', 'SI', 'UN']

2024-04-06 06:52:47,268 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2024-04-06 06:52:47,423 [INFO] updateprocess  Check if new report zip files are available...
2024-04-06 06:52:47,471 [INFO] updateprocess  check if there are new files to download from sec.gov ...
2024-04-06 06:52:48,147 [INFO] updateprocess  start to transform to parquet format ...
2024-04-06 06:52:48,164 [INFO] updateprocess  start to index parquet files ...


No rapid-api-key is set: 
If you are interested in daily updates, please have a look at https://rapidapi.com/hansjoerg.wingeier/api/daily-sec-financial-statement-dataset


## Finding the CIK for a company
The first interactive cell lets you search for a companies cik number by name.
For instance, just start to type apple.

In [7]:
# a simple way to find the cik for a company.
# e.g., start typing apple and watch the list get filtered
@interact(search_string=widgets.Text(value=''))
def search_cik(search_string):
    print(search_string)
    result_df = search.find_company_by_name(search_string)
    display(result_df)
    if len(result_df) > 0:
        print(result_df.cik.tolist()[0])

interactive(children=(Text(value='', description='search_string'), Output()), _dom_classes=('widget-interact',…

## Finding reports for a cik number
The next cell displays a list of availale reports for a company.<br>
First, enter the cik into the cik field. E.g., use apple's cik '320193'.<br>
Then chose the report types you want to filter for. The 10-K and 10-Q is preselected (annual and quarterly reports).<br>
Use the rows dropdown to configure how many rows shall be displayed.

In [8]:
# using a cik number, you can filter for all filed reports. Choose the type of report you want to see by selecting the entries in the forms widget.
# e.g., use apples cik '320193' to see an overview of the reports that have were filed by apple
@interact
def reports(cik=widgets.Text(value='0'), forms=widgets.SelectMultiple(options=forms_list, rows=6, value=['10-K', '10-Q']), rows=[10, 25, 50, 100]):
    reader = CompanyIndexReader.get_company_index_reader(cik=int(cik))
    reports = reader.get_all_company_reports_df(forms=list(forms))
    display(reports.head(rows))

interactive(children=(Text(value='0', description='cik'), SelectMultiple(description='forms', index=(4, 8), op…

## Showing the details of a report
Now we are ready to show the details of a report. <br>
<br>
Therefore, enter the report id in the adsh field. E.g. use '0000320193-22-000108' for the annual report of 2022 from apple.<br>
Use the stmts list to configure which which statements data should be shown. Per default, BS (balance sheet), IS (income statement), and CF (cash flow are activated). <br>
Select the number of displayed rows with the rows drop-down box.<br>
Sometimes, data is shown as positive even if the value is actually negative (or vice-versa). This is indicating by the negating flag. This is often the case in CF statements. There is a checkbox 'invert_negated' which switch the sign of number, if the have a value of one in the negating column. To see the effect, select only the CF stmt from the '0000320193-22-000108' report and swith the 'invert_negated' checkbox.<br><br>
**Note how fast the data is reloaded if you change the settings or display another report**  This is due to the fact, that we use the parquet format and a simple index.<br>
**Moreover, the data is sorted in the correct order as displayed in the original report.** Just click on the url in the 'Basic Information' to open the filed report directly at sec.gov and compare it.

In [4]:
@interact
def reports(adsh=widgets.Text(value='0'), stmts=widgets.SelectMultiple(options=stmt_list, rows=6, value=['BS', 'IS', 'CF']), rows=[50, 100, 200], invert_negated=widgets.Checkbox()):
    if adsh=='0':
        display(invert_negated)
        display('Nothing selected - Note: loading may take a few seconds')
        return
    display('loading...')
    reader = SingleReportCollector.get_report_by_adsh(adsh=adsh, stmt_filter=stmts)
    
    raw_data = reader.collect()
    filterd_data = raw_data.filter(ReportPeriodRawFilter())
    
    raw_stmts_data = filterd_data.filter(StmtRawFilter(stmts=stmt_list))

    report_data = filterd_data.join().present(StandardStatementPresenter(invert_negating=invert_negated))
        
    # get some key infos of the report
    submission_data = {k:v for k,v in raw_data.sub_df.loc[0].to_dict().items() if k in ['cik', 'adsh', 'name', 'cityba', 'form', 'period', 'filed']}

    # create and display the url on which the report is published on sec.gov, so that it can directly be opened    
    url = BaseReportIndexer.URL_PREFIX + str(submission_data['cik']) + '/' + submission_data['adsh'].replace('-','') + '/' + submission_data['adsh'] + '-index.htm'
    display(Markdown("## Basic Information"))
    display(url)

    # display the key submission data of the report
    display(submission_data)    
    
    display(Markdown("<br>"))
    
    # display the data of the report
    display(Markdown("## Details"))
    display(report_data.head(rows))
    
    display(Markdown("<br>"))
    display(Markdown("<br>"))
    display(Markdown("## Raw"))
    pre_def_sorted = raw_stmts_data.pre_df.sort_values(['report', 'line'])
    display(pre_def_sorted.head(rows))
    display(raw_stmts_data.num_df.head(rows))
    

interactive(children=(Text(value='0', description='adsh'), SelectMultiple(description='stmts', index=(0, 5, 1)…